# VARS-TOOL Tutorial

## Install

This is a simple tutorial show casing VARS-TOOL functionalities that are being implement in Python. For now, only single value functions can be fed to the package, however, the time-varying/-aggregate versions are to be added soon. Let's install the latest VARS-TOOL from GitHub:

In [23]:
!git clone https://github.com/vars-tool/vars-tool
!pip install vars-tool/.

Cloning into 'vars-tool'...


Processing c:\users\corde\giws\vars-tool\tutorial\vars-tool
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for varstool: filename=varstool-2.1-py2.py3-none-any.whl size=62317 sha256=8b8a7a5b13a4f48a88f3d24206b8cb61d51e3256f23a6918586f00e964ae9fac
  Stored in directory: c:\users\corde\appdata\local\pip\cache\wheels\ea\22\e6\94dfc4ddef885e03d97899daed3e63a6c2c3a1295f1b37f687
Successfully built varstool
  Attempting uninstall: varstool
    Found existing installation: varstool 2.1
    Uninstalling varstool-2.1:
      Successfully uninstalled varstool-2.1


## An example

Let's import what we have installed:

In [34]:
# importing VARS and Model from varstool package
from varstool import VARS, Model

import numpy as np
import pandas as pd

Let's define a sample function to use in the analysis:

In [25]:
def ishigami(x, a=7, b=0.05):
    '''Ishigami test function'''
    # check whether the input x is a dataframe
    
    if not isinstance(x, (pd.core.frame.DataFrame, pd.core.series.Series, np.ndarray, list)):
        raise TypeError('`x` must be of type pandas.DataFrame, numpy.ndarray, pd.Series, or list')
    
    if len(x) > 3:
        raise ValueError('`x` must have only three arguments at a time')
    
    return np.sin(x[0]) + a*(np.sin(x[1])**2) + b*(x[2]**4)*np.sin(x[0])

The `Model` class is a wrapper for custome functions in the online version of VARS. However, the first argument of every function must accept an array of parameters:

In [30]:
ishigami_model = Model(ishigami)

Create `experiment_1` and initialize the values needed to run a VARS analysis:

In [35]:
experiment_1 = VARS(parameters = {'x1':(0, 1), 'x2':(0, 1), 'x3': (0, 1)},
                    delta_h = 0.1,
                    ivars_scales = (0.1, 0.3, 0.5),
                    num_stars=10,
                    sampler = 'rnd',
                    model = ishigami_model,
                    bootstrap_flag = True,
                    bootstrap_size = 100,
                    bootstrap_ci=0.9,
                    grouping_flag=True,
                    num_grps=2
                )

We can get a status report of the analysis by typing the instance class name:

In [36]:
experiment_1

AttributeError: 'VARS' object has no attribute 'num_grps'

Let's run the online version - please note that for now we do not have a progress report (under investigation):

In [37]:
experiment_1.run_online()
experiment_1

AttributeError: 'Model' object has no attribute '__name__'

And different values are accessible:

In [9]:
experiment_1.variogram_value.unstack(0)

param,x1,x2,x3
h,,,
1,0.003745,0.154738,0.000003
2,0.015089,0.632787,0.000009
3,0.034099,1.441989,0.000016
4,0.060719,2.570765,0.000023
5,0.094762,3.986608,0.000031
6,0.135913,5.636381,0.000040
7,0.183732,7.448499,0.000053
8,0.237660,9.336946,0.000072
9,0.297025,11.206792,0.000103


In [10]:
experiment_1.covariogram_value.unstack(0)

param,x1,x2,x3
h,,,
1,2.534029,2.117666,2.362398
2,2.518026,1.437912,2.362441
3,2.498901,0.610013,2.362432
4,2.476827,-0.335223,2.362375
5,2.452015,-1.360884,2.362268
6,2.424710,-2.425383,2.362111
7,2.395192,-3.484091,2.361898
8,2.363767,-4.491120,2.361624
9,2.330773,-5.401187,2.361282


In [11]:
experiment_1.sobol_value

param
x1    0.020646
x2    0.859436
x3    0.000006
dtype: float64

In [12]:
experiment_1.morris_value[0].unstack(0)

param,x1,x2,x3
h,,,
1,0.085613,0.525746,0.001307
2,0.172244,1.077891,0.002405
3,0.259459,1.645587,0.003383
4,0.346823,2.217368,0.004332
5,0.433894,2.781451,0.005340
6,0.520234,3.326053,0.006498
7,0.605405,3.839704,0.007895
8,0.688975,4.311563,0.009620
9,0.770521,4.731710,0.011763
